In [1]:
import pandas as pd
import numpy as np
import time
import pickle
import re

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

import xgboost as xgb

rand_state = 1337

### Define pipelines and functions 
1. Preprocessing 2. Sampling 3. Scoring

In [2]:
# Pipelines: Defining the categorical imputation and one-hot encoder for categorical variables.
categorical_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent"))
    ]
)

# defining the numerical imputation and standard scaler for numerical variables.
numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), 
           ("scale", StandardScaler())]
)

# def_prep_df: Preparing the TRAINING data for creating and testing the model.
def prep_df(df, target, target_to_drop):

    # save indices
    df_index = df.index
    # save statement_age column
    statement_age_s = df['statement_age']

    # Drop columns that shouldn't be scaled or imputed
    df = df.drop(columns=["s_2", 'statement_age', target_to_drop])

    # Missing values handling
    missing_props = df.isna().mean(axis=0)
    

    over_threshold = missing_props[missing_props >= 0.4]
    over_threshold
    

    df.drop(over_threshold.index, axis=1, inplace=True)

        
    # Split into predictors and target
    X = df.drop(columns=[target])
    y = df[target]

    # Split categorical and numerical columns
    cat_cols_all = ['b_30', 'b_38', 'd_114', 'd_116', 'd_117', 'd_120', 'd_126', 'd_63', 'd_64', 'd_66', 'd_68', 'b_31', 'd_87']
    cat_cols = [col for col in X.columns.str.lower() if col in cat_cols_all]
    num_cols = [col for col in X.columns.str.lower() if col not in cat_cols]
    
    # get dummies for categorical variables
    Xcat = pd.get_dummies(X[cat_cols], columns=cat_cols, drop_first=True)
    
    X = pd.concat([X[num_cols],Xcat], axis=1)
    X.columns = X.columns.str.lower()
    cols_list = X.columns.tolist()

    cat_cols = [col for col in cols_list if col not in num_cols]
   


    full_processor = ColumnTransformer(
        transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
        ]
    )


    
    # Apply preprocessing
    X_processed = full_processor.fit_transform(X)
    X_processed = pd.concat([pd.DataFrame(X_processed, index=df_index), statement_age_s], axis=1)
    print(X_processed.shape)

    y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
            y.values.reshape(-1, 1)
            )
    y_processed = pd.DataFrame(y_processed, index=df_index)

    
    return X_processed, y_processed, cols_list




def get_train_test(df_train, df_train_y, X_processed, y_processed, usefraction):
    n = 100
    ids = np.array(df_train_y.index)
    target = np.array(df_train_y['target'])
    
    skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=rand_state)
    skf.get_n_splits(ids, target)

    i = 0
    id_subsets = [None]*n
    for _, subset in skf.split(ids, target):
        id_subsets[i] =list(ids[subset])
        i += 1

    
    list1 = list(np.arange(0, int(usefraction[0]*100), 1))
    list2 = list(np.arange(int(usefraction[0]*100), int(usefraction[0]*100)+int(usefraction[1]*100), 1))


    train_ids = []
    for i in list1:
        train_ids.extend(id_subsets[i])
    test_ids = []
    for i in list2:
        test_ids.extend(id_subsets[i])


    X_train = X_processed[df_train.index.isin(train_ids)]
    y_train = y_processed[df_train.index.isin(train_ids)]
    X_test = X_processed[df_train.index.isin(test_ids)]
    y_test = y_processed[df_train.index.isin(test_ids)]


    print(f'Train data obs.: {len(X_train)}')
    print(f'Test data obs: {len(X_test)}')

    return X_train, X_test, y_train, y_test




def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

### Create initial df to be further processed

In [3]:
df_train_x = pd.read_parquet('./../ignore/train.parquet')
df_train_x.columns = df_train_x.columns.str.lower()
df_train_x = df_train_x.sort_values(['customer_id', 's_2'])
df_train_x = df_train_x.set_index('customer_id')

df_train_y = pd.read_csv('./../ignore/train_labels.csv')
df_train_y.columns = df_train_y.columns.str.lower()
df_train_y = df_train_y.set_index('customer_id')



df_train = pd.merge(df_train_x, df_train_y, left_index=True, right_on='customer_id', how='left')

df_train['last_statement_flag'] = (df_train.groupby(df_train.index)['s_2']
                      .rank(method='dense', ascending=False)
                      .astype(int)
                   )

df_train['last_statement_target'] = df_train['target']*df_train['last_statement_flag'].apply(lambda x: 1 if x==1 else 0)
df_train = df_train.rename(columns={'last_statement_flag':'statement_age'})

### Select which statements to use

In [4]:
use_statements = [1,2,3]
df_train = df_train[df_train['statement_age'].isin(use_statements)]

### Process all the data after selecting statements

In [5]:
# Prep the dataframe
# Note that the last column 'statement_age' is left in the dataframes for scoring, not for predicting!
X_processed, y_processed, cols_list = prep_df(df_train, target='target', target_to_drop='last_statement_target')

(1360401, 204)


### Get samples for training and testing

In [6]:
# First vale of "usefraction" specifies the train size and the second, the test size (fraction of total train data available)
X_train, X_test, y_train, y_test = get_train_test(df_train, df_train_y, X_processed, y_processed, usefraction = [0.7, 0.3])

Train data obs.: 952359
Test data obs: 408042


## XGB
feature selection loop using feature importances

In [7]:
feature_reduction_rate = 0.15 # Attempt to remove 25% of remaining features in each loop iteration
accuracy_loss = 0.001 # accepted accuracy loss c.f. max accuracy

scores = []
feature_ind = [list(range(0,(X_train.shape[1]-1)))]
feature_names = [cols_list]
remove_n_features = [int(len(feature_ind[0])*feature_reduction_rate)]
i = 0


def get_reduced_features(xgbc, feature_names, remove_n_features):

    xgb_feature_imp = pd.DataFrame({'name':feature_names,
                                    'importances':[val[0] for val in xgbc.feature_importances_.reshape(-1,1)]})

    xgb_feature_imp = xgb_feature_imp.sort_values(by='importances', ascending=False)
    
    df_features = xgb_feature_imp.iloc[:-remove_n_features,:]
    feature_ind = df_features.index
    feature_names = df_features['name']


    return feature_ind, feature_names



def fit_predict(feature_ind, feature_names, remove_n_features, X_train, y_train, X_test, y_test):
    xgbc = xgb.XGBClassifier(use_label_encoder=False).fit(X_train[feature_ind], y_train, verbose=0, eval_metric='logloss')
    
    y_pred_a_xgb = pd.DataFrame({'customer_id':X_test.index.values,
                            'statement_age':X_test.iloc[:,-1].values,
                            'prediction':[val[1] for val in xgbc.predict_proba(X_test[feature_ind])]})
    
    last_proba_xgb = y_pred_a_xgb[y_pred_a_xgb['statement_age']==1].set_index('customer_id')
    score = [amex_metric(y_test.groupby(y_test.index).max().rename(columns={0:'target'}), last_proba_xgb)]

    feature_ind, feature_names = get_reduced_features(xgbc, feature_names, remove_n_features)

    return score, feature_ind, feature_names, xgbc




while True:
    start_time = time.time()

    cscore, cfeature_ind, cfeature_names, cxgb = fit_predict(feature_ind[i], feature_names[i], remove_n_features[i], X_train, y_train, X_test, y_test) # "c" denotes current
    scores += cscore # add score to list

    if i > 0:
        if (max(scores) - scores[i]) >= accuracy_loss: # Maximum residual between max and current score
            
            print(f'Iter {i+1}, First fit yielded too low score, trying again (number of features attempted: {len(feature_ind[i])}')
            cscore, cfeature_ind, cfeature_names, cxgb = fit_predict(feature_ind[i], feature_names[i], remove_n_features[i], X_train, y_train, X_test, y_test) # "c" denotes current

            if (max(scores) - cscore) >= accuracy_loss: # use same criterion again
                
                # print(f'Iter {i+1}, Duration: {round((time.time() - start_time),3)} s, Iteration failed, too large accuracy loss ({cscore-max(scores)}), removing fewer features')
                print(f'Iter {i+1}, Duration: {round((time.time() - start_time),3)} s, Score: {round(scores[i],5)} - Iteration failed, too large accuracy loss ({cscore-max(scores)})')
                feature_ind += [feature_ind[i-1]] # add list of feature indices to list
                feature_names += [feature_names[i-1]] # add list of feature names to list

                feature_reduction_rate = feature_reduction_rate*0.5 # decrease feature reduction rate if iteration failed

                if int(len(feature_ind[i])*feature_reduction_rate) >= 1: # Check that at least 1 feature can be removed
                    remove_n_features += [int(len(feature_ind[i])*feature_reduction_rate)]
                    print(f'Current feature reduction rate: {feature_reduction_rate}')
                    i += 1
                    continue
                else:
                    print('Completed')
                    break


            else:
                scores[i] = cscore # Overwrite first fit score with 2nd fit score
                print(f'Iter {i+1},  Retry successful')



    feature_ind += [cfeature_ind] # add list of feature indices to list
    feature_names += [cfeature_names] # add list of feature names to list
    remove_n_features += [int(len(feature_ind[i])*feature_reduction_rate)]
    
    print(f'Iter {i+1}, Duration: {round((time.time() - start_time),3)} s, Score: {round(scores[i],5)}, Number of features: {len(feature_ind[i])}')

    i += 1

Iter 1, Duration: 221.322 s, Score: 0.77797, Number of features: 203
Iter 2, Duration: 213.88 s, Score: 0.77827, Number of features: 173
Iter 3, Duration: 185.148 s, Score: 0.77926, Number of features: 143
Iter 4, Duration: 166.304 s, Score: 0.78018, Number of features: 118
Iter 5, First fit yielded too low score, trying again (number of features attempted: 97
Iter 5, Duration: 275.991 s, Score: 0.77705 - Iteration failed, too large accuracy loss ([-0.00312886])
Current feature reduction rate: 0.075
Iter 6, Duration: 167.509 s, Score: 0.78018, Number of features: 118
Iter 7, First fit yielded too low score, trying again (number of features attempted: 111
Iter 7, Duration: 307.593 s, Score: 0.7789 - Iteration failed, too large accuracy loss ([-0.00128165])
Current feature reduction rate: 0.0375
Iter 8, Duration: 162.897 s, Score: 0.78018, Number of features: 118
Iter 9, First fit yielded too low score, trying again (number of features attempted: 114
Iter 9, Duration: 324.193 s, Score: 0

In [8]:
savelist = list(feature_names[len(feature_names)-2])

# remove the suffix from dummy variables and make sure the list elements are unique
savelist = list(np.unique(np.array([re.findall(r'(\w_\d+)', x)[0].upper() for x in savelist])))

f = open('features2.pickle', 'wb')
pickle.dump(savelist, f)
f.close()

In [ ]:
f = open('features.pickle', 'rb')
loadlist = pickle.load(f)
f.close()